![CH6-ADS.png](.\Media\CH6-ADS.png)

# <span style="color:#cc5500;">Dedicated Administrators Connection: The DAC</span>

## <span style="color:#cc5500;">What is the DAC?</span>

The DAC is nothing more than the acronym associated with the Dedicated Administrators Connection.  The DAC was created as a special tool for DBA's to connect to SQL Server and run queries in cases of critical performance conditions and problems.  SQL Server reserves a thread for you to connect to an instance and run queries in an emergency.   The DAC and the Remote DAC are essentially the same with the difference being 'where' you connect.  The DAC is enabled locally by default, but since most DBA's do not install SQL Server Managment Studio on their servers, I will demonstrate using the Remote DAC from a management server.

SQL Server provides a special diagnostic connection for administrators when standard connections to the server are not possible. This diagnostic connection allows an administrator to access SQL Server to execute diagnostic queries and troubleshoot problems even when SQL Server is not responding to standard connection requests.  

This dedicated administrator connection (DAC) supports encryption and other security features of SQL Server.  The DAC only allows changing the user context to another admin user.  

SQL Server makes every attempt to make DAC connect successfully, but under <u>extreme</u> situations it may not be successful.  



## <span style="color:#cc5500;">Understand some restrictions of the DAC?</span>

Because the DAC exists solely for diagnosing server problems in rare circumstances, there are some restrictions on the connection:

- To guarantee that there are resources available for the connection, only one DAC is allowed per instance of SQL Server. If a DAC connection is already active, any new request to connect through the DAC is denied with error 17810.
- To conserve resources, SQL Server Express does not listen on the DAC port unless started with a trace flag 7806.
- The DAC initially attempts to connect to the default database associated with the login. After it is successfully connected, you can connect to the master database. If the default database is offline or otherwise not available, the connection will return error 4060
- However, it will succeed if you override the default database to connect to the master database instead using the following command:   sqlcmd -A -d master
- SQL Server prohibits running parallel queries or commands with the DAC. For example, error 3637 is generated if you execute either of the following statements with the DAC:
    - RESTORE Database
    - BACKUP Database
- Only limited resources are guaranteed to be available with the DAC. Do not use the DAC to run resource-intensive queries (for example. a complex join on large table) or queries that may block. This helps prevent the DAC from compounding any existing server problems.
- For more information about these restrictions, see this link: [Diagnostic Connection for Database Administrators - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/database-engine/configure-windows/diagnostic-connection-for-database-administrators?redirectedfrom=MSDN&view=sql-server-ver15#restrictions)

## <span style="color:#cc5500;">When to use the DAC and how do I connect?</span>

Plain and simple, the DAC is for emergencies.

The Remote DAC is very important.  If for some reason you can't RDP to your SQL Management Server, if you cannot connect, or you can use the Remote DAC.  Because it is a single-port, single-thread, low memory, reserved connection, and cannot make use of parallel operations, do not use the DAC for every day administrative functions like checking in on the status of your backup jobs or index rebuild status.

The first thing to do is see if the DAC is enabled, if not, we will enable it.

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
--  Check for any pending configurations before you start.  If value_in_use is set to 1, then it is enabled
SELECT *
FROM sys.configurations
WHERE name = 'remote admin connections'
GO

In [ ]:
-- If it wasn't enable from the query results above, enable it now
EXEC sp_configure 'remote admin connections', 1;
GO
RECONFIGURE

## <span style="color:#cc5500;">Make a DAC Connection with SSMS</span>

There are two ways to connecto to the Remote DAC.  1) Through a New Query in SSMS and 2) from the Command Prompt

If can connect to an installation of SQL Server Management Studio that has the ability to connect to your cripled SQL Server, then you can run your queries through the SSMS User Interface.  If so, you will likely find this a better experience than using a Command Prompt.

In SSMS select File New Database Engine Query

![CH6-NewAdminQuery.jpg](.\Media\CH6-NewAdminQuery.jpg)

Then ender admin: in front of the fully qualified Instance name

![CH6-AdminConnect.jpg](.\Media\CH6-AdminConnect.jpg)

It should connect you to the Instance.  Notice that at the bottom of your Query window, you will see admin: displayed in your connection.  If you have difficulty making a connection with a New Query, you may need to temporarily enable the SQL Browser Service.  If you receive a warning that someone else is already connected with a DAC connection, you can run this query to see who it is.

In [ ]:
SELECT
    a.host_name,
    a.original_login_name,
    a.session_id,
    a.[status]
FROM sys.dm_exec_sessions as a 
JOIN sys.endpoints as b 
on a.endpoint_id = b.endpoint_id
where b.name = 'dedicated admin connection'

In [ ]:
--Now that you are connected in SSMS copy this query and paste it into your SSMS Admin Query

EXEC xp_ReadErrorLog 0, 1, N'Error'

View your resuls in SSMS

![CH6-Error.jpg](.\Media\CH6-Error.jpg)


In [ ]:
--Now that you are connected in SSMS copy this query and paste it into your SSMS Admin Query
-- Get information on location, time and size of any memory dumps from SQL Server (Memory Dump Info)
SELECT [filename], creation_time, size_in_bytes/1048576.0 AS [Size (MB)]
FROM sys.dm_server_memory_dumps WITH (NOLOCK) 
ORDER BY creation_time DESC OPTION (RECOMPILE);

## <span style="color:#cc5500;">Make a DAC Connection from a Command Prompt</span>

What if, for some reason, you are not able to RDP to your jump box where SSMS is installed?  You are not out of options.  You can connect from the command prompt.   Command Line syntax for making SQLCMD connections to your SQL instance is as follows:

Syntax - "sqlcmd" followed by paramteters.  -S is the Severname (your fully qualified Instance), -U user name if using a SQL  account, -P your password, -d is the database (prefer master) -A means to make an Dedicated Admin Connection

Using a SQL Account:  

- sqlcmd -S KEVINWARRENSB2\\VIPER -U sa -P MyHardP@ssw0rd -d master -A

Using a DOMAIN Account:

- sqlcmd -S KEVINWARRENSB2\\VIPER -d master -A

If you do not see your Command Prompt Terminal (by default at the bottom of your screen) from your menu bar, select View \> Terminal

Enter your connection command

![CH6-CMDConnect.jpg](.\Media\CH6-CMDConnect.jpg)

It should connect you to that instance.  You will know you are connected if you do not get an error and you are presented with the 1\>   prompting you to enter your next Command

Let's enter our first SQL statement to check on the health of this instance.  Say, for example you are having an emergency with your SQL Server and you want to look at your transaction log space.

In [ ]:
-- Don't run this command in this code block.  I've pasted it in here to make it easier for you to copy the statement
-- Copy this command and paste it into your command terminal with your DAC connection
-- Remember to add a GO below your statement, if not the command will not run until you add the GO and then hit enter on your keyboard
DBCC SQLPERF(LOGSPACE);  
GO

![CH6-LogSpace.jpg](.\Media\CH6-LogSpace.jpg)

## <span style="color:#cc5500;">What is next?</span>

What is next is really up to you!  Making a DAC connection is one of those things that you don't do often.

- Return to this notebook for a refresher
- Practice makes perfect.  Unless you practice making a DAC connection, you may lose valuable time in an emergency.  Rehearse often
- Take some time, to create a set of emergency query's that you can pull out in an emergency and save them somewhere easily accessible.  (many of those emergency query's are already provided in other chapters of this notebook)
- Remember the restrictions I posted above.  You don't want to use the DAC to run an extremely resource consuming query and only make the situation worse.

Here's a link that shows the SQLCMD utility and command: [sqlcmd Utility - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-us/sql/tools/sqlcmd-utility?view=sql-server-ver15)